# Transformers

this note book is here to help me refresh some of my understanding of the basic transformers architecture

we want to implement the encoder part of the architecture in [attention is all you need paper](https://arxiv.org/pdf/1706.03762):




architecture screentshot:

![](20251120024008.png)

My goal with be to go through one pass of transformer layer for a data, and try to explain each layer, finally I will convert this jupyter notebook to a python code and train it on a simple dataset

In [1]:
# I want this note book to be very simple so I will make the data very simple, i.e use whatever I have written till now as training data

training_data = list("""
# Transformers

this note book is here to help me refresh some of my understanding of the basic transformers architecture

we want to implement the encoder part of the architecture in [attention is all you need paper](https://arxiv.org/pdf/1706.03762):

My goal with be to go through one pass of transformer layer for a data, and try to explain each layer, finally I will convert this jupyter notebook to a python code and train it on a simple dataset

# I want this note book to be very simple so I will make the data very simple, i.e use whatever I have written till now as training data

""")

In [2]:
# I don't want to get too deep into tokenization for this notebook so I am just going to instead use all the unique characters
# present in the training data as distinct tokens
vocabulary_list = list(set(training_data))

In [3]:
print(vocabulary_list[:5])
print(len(vocabulary_list))

['k', '#', '[', 'g', ':']
44


In [4]:
# let's create training and testing data
# training and testing data for next token prediction would look something like

# the way the transformer works is that for a single example sentence it trains the model for multiple token prediction
print(training_data[:9])

['\n', '#', ' ', 'T', 'r', 'a', 'n', 's', 'f']


In [5]:
# here if x is
training_data[:8]

['\n', '#', ' ', 'T', 'r', 'a', 'n', 's']

In [6]:
# then y would be
training_data[1:9]

['#', ' ', 'T', 'r', 'a', 'n', 's', 'f']

In [7]:
# ok before we make create training data we need to convert our tokens to a unique index to do that I will do
token_to_index = {c:i for i,c in enumerate(vocabulary_list)}
index_to_token = {i:c for i,c in enumerate(vocabulary_list)}

In [8]:
# now we let's convert our training data to a torch tensor
import torch

training_data_tensor = torch.tensor([token_to_index[c] for c in training_data], dtype=torch.long)

In [9]:
print(training_data_tensor[:10])
print([index_to_token[ix.item()] for ix in training_data_tensor[:10]])

tensor([26,  1, 30, 13, 36, 41, 34, 38, 28,  9])
['\n', '#', ' ', 'T', 'r', 'a', 'n', 's', 'f', 'o']


In [10]:
# now let's create training and testing set
block_size = 8
x = torch.stack([training_data_tensor[ix:ix+block_size] for ix in range(len(training_data_tensor)-block_size)] )
# max ix len(training_data_tensor)-block_size - 1
# so ix + block_size = len(training_data_tensor) - 1
# so final example won't include last character
y = torch.stack([training_data_tensor[ix:ix+block_size]for ix in range(1,len(training_data_tensor)-block_size+1)]) 



In [11]:
print("x training data")
print(x[:5])
print("y training data")
print(y[:5])

x training data
tensor([[26,  1, 30, 13, 36, 41, 34, 38],
        [ 1, 30, 13, 36, 41, 34, 38, 28],
        [30, 13, 36, 41, 34, 38, 28,  9],
        [13, 36, 41, 34, 38, 28,  9, 36],
        [36, 41, 34, 38, 28,  9, 36, 10]])
y training data
tensor([[ 1, 30, 13, 36, 41, 34, 38, 28],
        [30, 13, 36, 41, 34, 38, 28,  9],
        [13, 36, 41, 34, 38, 28,  9, 36],
        [36, 41, 34, 38, 28,  9, 36, 10],
        [41, 34, 38, 28,  9, 36, 10, 11]])


# Embedding Table

![](20251121001141.png)


This is a look up table between the vocabulary index and n dimensional vector,
during the training of transformer model this vectors also gets trained, i.e where these vectors point to gets updated,
based on the similarity between these vectors, if let's say I have 2 tokens "dog" and "pooch", during the start of training process
they might point in very different directions, but after the training both would point to pretty much same place

### Question?:

1. What is so special about the training process that transforms these vectors from pointing in random ass direction, to actually have some meaning
    * for now I am gonna assume that the answer is that the transformer architecture expects and assumes these vectors to be what I have described
    * and based on this assumption, the subsequent layers performs its operation, so optimizing the loss leads to these embedding vector looking more like actual high dimensional representation of the words 

In [12]:
from torch import nn

EMBEDDING_DIMENSION = 8
VOCAB_SIZE = len(vocabulary_list)

embeddings_table = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIMENSION)

In [13]:
# some experimentation on how embeddings table work,
print(embeddings_table(torch.tensor([[0,1,2,3]], dtype=torch.long)))
# it goes to each item in tensor and assumes each item is a index converts it to its corresponding embedding vector

tensor([[[-1.0245,  0.0273, -0.7931, -0.7083, -0.8083, -0.6705,  1.4848,
           1.0309],
         [-1.9201,  0.2062,  0.3763, -2.1295, -0.3714, -0.2258, -0.1815,
          -1.3935],
         [-0.5931,  1.2472,  1.3167, -1.5546,  1.2499,  0.7978,  0.2492,
           0.4915],
         [ 1.0728,  0.5471, -1.3715,  0.1125,  1.5603,  0.3472, -1.7151,
           1.0866]]], grad_fn=<EmbeddingBackward0>)


I want to do a very simple forward pass so I am gonna create my forward pass batch now

In [14]:
x_batch = x[:5]
y_batch = y[:5]

A question lingers, what does this (shifted right) mean:

![](20251121002201.png)

this just means that our input is shifted from the target output

In [15]:
x_embeddings = embeddings_table(x_batch)

In [16]:
# just one example
x_embeddings[:1]

tensor([[[ 0.4142, -0.5659,  0.9072,  2.7817, -1.0277, -0.0627, -1.2612,
          -1.7454],
         [-1.9201,  0.2062,  0.3763, -2.1295, -0.3714, -0.2258, -0.1815,
          -1.3935],
         [-1.1141, -1.2467, -2.1987, -1.0727, -0.4632,  0.2007,  0.6056,
          -0.2207],
         [-2.4173,  0.0631, -0.5981,  0.6192,  0.0318, -0.4283, -0.1021,
           0.0426],
         [-0.1295,  0.2368, -0.6105,  1.2359,  1.3012,  0.0046, -0.1444,
          -0.2830],
         [ 0.7166, -0.0982, -0.8280,  0.2714,  1.2153,  0.6304, -1.3637,
          -1.2442],
         [ 0.8068, -0.0297, -0.3025, -0.8191, -1.2846,  0.7870,  0.2980,
           1.0460],
         [-1.2761,  0.2939, -0.2014,  1.6352, -0.3573,  0.2533, -0.9804,
           1.5339]]], grad_fn=<SliceBackward0>)

# Positional Encoding

![](20251121135418.png)


From my past understanding this is sort of values with varies with the position of the token in the sequence to encode the information about the position of the token in the sequence

so for each position there will be a vector associated to it, which will get added to the original embedding vector at that position

### Questions?:
1. Why Add these vectors to the original embedding vector? Can it not be appended or create some other type of encoding create a new channel perhaps like we do for CNNs
    - Ans: The Idea behind adding these is how we treat embedding vectors, you can think of embedding vector as the original absolute meaning of a token, now depending on whether it appears at the beggining of a sentence or end of a sentence it's meaning might differ, i.e its embedding vector might change its position, that change is capture by the addition of this positional embeddding vector
2. Why do these needs to be a vector all together can these not be like a single number which gets added?
    - Ans: well a vector is a more generalized version of a single number, if single number is the right approach then expectation is that the network would train the embedings to become a single number

## Sinusoidal Encoding

![](20251121140214.png)

here d_model is the dimension of the embedding

In the original Paper they used a fix positional sinusoidal encoding, they mentioned the performance for both learned and not learned were identical, they wanted to experiment with sinusoidal encoding, because they wanted to test the model beyond the trained context length

# Question?:
1. But why sinusoidal encoding

In [17]:
positional_embedding_table = nn.Embedding(block_size, EMBEDDING_DIMENSION)

In [18]:
x_pos_embeddings = positional_embedding_table(torch.arange(x_embeddings.shape[1])) # C, E

x_embeddings # B, C, E

x_embeddings_total = x_embeddings + x_pos_embeddings # B, C, E + C, E -> pytorch checks the shape starting from right and if there is an extra dimension it creates a new dimention and copuies the same thing over, like C, E -> (1, C, E) -> (B, C, E)

# Self Attention Layer

![](20251123232032.png)

I will start of by explaining what this layer does in a high level, then I will dig deep into how it does this, initially I will go over a single head self attention, 
then understand myself and explain why multi head self attention

this is the 3b1b interpretation of this layer on a high level, which I found to be the most elegant

## The Explanation
This layer as a whole tells us how should the original embedding vector be modified, so that it's meaning is enriched with the context of the surrounding tokens, for example take the sentence:

" That blue aeroplane is very dangerous "

in this example initially "aeroplane"'s embedding vector would straight up point to the absolute aeroplane,
then attention layer outputs a result, that result when added to the original embedding vector, nudges the aeroplane's vector in a direction closer to blue and dangerous

that is on high level what this layer does, now going into the detail let's start by the equation

![](20251123233808.png)

the above represent the equation describing the self attention mechanism, for the purpose of this excercise we will focus on masked self attention

here Q, K, V are all matrices

Q being the query matrix, K Key matrix and V value matrix

let me do one thing and form this forumla in our on going example and then explain

In [19]:
d_k = d_q = 4

W_q = nn.Linear(EMBEDDING_DIMENSION, d_q, bias = False)
W_k = nn.Linear(EMBEDDING_DIMENSION, d_k, bias = False)
W_v = nn.Linear(EMBEDDING_DIMENSION, EMBEDDING_DIMENSION, bias = False) # for single head attention this needs to be same as embedding dimension cause the resulting vector of attention layer get added to input embedding vector so both need to have same dimension


In [20]:
W_q.weight.shape

torch.Size([4, 8])

In [21]:
x_embeddings_total.shape

torch.Size([5, 8, 8])

In [22]:
W_q.weight@x_embeddings_total[0][0]

tensor([ 0.7085,  0.0792, -0.8448,  1.7425], grad_fn=<MvBackward0>)

In [23]:
W_q(x_embeddings_total)

tensor([[[ 0.7085,  0.0792, -0.8448,  1.7425],
         [-1.0584,  1.2724,  1.1425, -0.8683],
         [ 0.9038,  0.1175,  0.6418, -0.0202],
         [-0.3416,  1.0646,  1.5527,  0.4360],
         [-1.2428,  0.6257,  0.7662,  2.0631],
         [-0.2865,  0.3382, -1.0810,  1.1033],
         [-0.4641, -0.9382,  1.1327, -0.2695],
         [-0.7241,  0.9603,  0.4288,  0.7924]],

        [[-0.8181,  0.9673,  1.5647,  0.2710],
         [ 0.2616,  0.0624,  1.3231, -0.3658],
         [ 0.3133,  0.7040,  0.3463,  0.2569],
         [-0.3125,  0.8173,  0.8526,  1.2972],
         [-1.3711,  0.9055,  0.3843,  2.4687],
         [-0.0335, -0.3776, -0.4019, -1.1157],
         [-0.5875, -0.4943,  1.1131,  0.7643],
         [-0.3391, -0.1131, -0.1430,  0.3208]],

        [[ 0.5019, -0.2427,  1.7453,  0.7735],
         [-0.3289,  0.6489,  1.0276, -0.0887],
         [ 0.3424,  0.4567, -0.3538,  1.1181],
         [-0.4407,  1.0970,  0.4707,  1.7028],
         [-1.1181,  0.1896,  1.0633,  0.2497],
         

In [24]:
# pass the existing vector through a trainable linear transformation

Q, K, V = W_q(x_embeddings_total), W_k(x_embeddings_total), W_v(x_embeddings_total)

In [25]:
print(Q.shape, K.shape, V.shape)

torch.Size([5, 8, 4]) torch.Size([5, 8, 4]) torch.Size([5, 8, 8])


In [26]:
attention_matrix = Q@K.transpose(1, 2) # transpose the 1st and the 2nd dimension not the  this is equivalent to Q[i]@K[i].transpose where i is each element of the batch

Let me try and explain what just happened above, let's take example of a single batch

In [27]:
print(Q[0][:3])
print(K[0][:3])

tensor([[ 0.7085,  0.0792, -0.8448,  1.7425],
        [-1.0584,  1.2724,  1.1425, -0.8683],
        [ 0.9038,  0.1175,  0.6418, -0.0202]], grad_fn=<SliceBackward0>)
tensor([[ 0.0921, -0.2833,  0.5043,  0.1507],
        [-1.7944, -1.0813,  0.8487,  0.3863],
        [-2.3796, -0.0154,  0.8617,  1.2738]], grad_fn=<SliceBackward0>)



well the traditional explanation is:

when the static embedding for a word is passed through these layers it extract specific feature pertaining to corresponding transformation

let's take an example: "The bank of the River"

Query -> transform the static embedding of bank to something like "I am a Noun needing a definition, need to know what am I a river bank, a financial bank, etc"

Key -> transforms the static embedding of River to something like "I am a nature related word, related to river"

Value -> transforms the embedding to actual meaning info, as their it might not be the first attention layer, if it is a second layer, it won't be the absolute meaning

but this never really sat with me completely, I was not able to understand this fully and it seemed pretty handwavy as to explaining what these layers really do, maybe I understand for Value vector but not for Query and Key vector

need to think and understand this properly and clearly once and for all, 
think, what would happen if no linear layer was present


ok let's say no linear layer was present, than the Q*K.T would just give me the cosine similarity between static embeddings of tokens

let's say the attention matrix is

`A`

here `A[i][j]` would be the cosine similarity between the ith token and jth token  (here when I say token I mean the embedding of token)

now let's see what would `A@V` do, here I am assuming `V` to is just the static emebdding


`A_softmax` -> is softmax function applied on `A` across each column (why column? cause weights are distributed across column see below)

```

V`[i] = V[0]*A_softmax[i][0] + V[1]*A_softmax[i][1] + V[2]*A_softmax[i][2] + ...

```

here `V[i]` represents the static embedding for the ith word, so `V[i]` is a vector, and ``V`[i]`` is the final vector after attention matrix multiplication

``V`[i]`` we can think of it as change or how the original word gets modified

so when taking the weighted sum it tells us, in direction of which static embedding should the input embedding move to the most and least (depending on the weight), so that it represents that modified vector would represent the meaning of the entire sentence, **now I see the poblem with just using static embedding, the problem is if we just use static embedding to compute cosine similarity it would nudge the input emebedding to this layer in the direction which is most similar to the word which we see in the sentence, which we don't always want**

side note: one more thing I learned is that these query and key vector live in a space which is smaller than the static embedding dimension, so it would help computationaly too

In [28]:
# now each element of this of shape context length x context length
attention_matrix.shape

torch.Size([5, 8, 8])

In [29]:
attention_matrix[0]

tensor([[-1.2065e-01, -1.4007e+00, -1.9547e-01, -1.1356e+00, -3.3180e-01,
          1.6046e+00, -6.1904e-01, -9.4741e-01],
        [-1.2603e-02,  1.1576e+00,  2.3776e+00,  2.2315e+00,  1.7283e+00,
          4.2887e-01, -5.6663e-01,  2.1490e+00],
        [ 3.7054e-01, -1.2120e+00, -1.6253e+00,  5.1343e-01,  9.6979e-01,
          4.3328e-01, -1.4799e-02,  3.6577e-01],
        [ 5.1572e-01,  9.4807e-01,  2.6899e+00,  1.8548e+00,  3.0317e+00,
          2.0422e+00, -4.0530e-01,  1.8306e+00],
        [ 4.0566e-01,  3.0009e+00,  6.2361e+00,  4.0957e-01,  2.6862e+00,
          3.1695e+00, -2.3296e-01,  7.4688e-01],
        [-5.0109e-01, -3.4279e-01,  1.1505e+00, -8.1284e-01, -9.3601e-01,
          8.7861e-01, -7.5810e-01, -5.8079e-01],
        [ 7.5372e-01,  2.7043e+00,  1.7514e+00,  6.7603e-02,  1.3870e+00,
         -2.9767e-01,  1.4286e+00,  3.1185e-02],
        [-3.0405e-03,  9.3107e-01,  3.0873e+00,  8.8694e-01,  1.4680e+00,
          1.7499e+00, -6.9229e-01,  1.0234e+00]], grad_fn=<Select

```

V`[i] = V[0]*A_softmax[i][0] + V[1]*A_softmax[i][1] + V[2]*A_softmax[i][2] + ...

```

now in self attention when we compute the change required for ith vector we don't want any token above ith token to be deciding how it changes, cause during inference we don't have that information at all so we would need to find a way so that above becomes

```

V`[i] = V[0]*A_softmax[i][0] + V[1]*A_softmax[i][1] + V[2]*A_softmax[i][2] + ... + V[i]*A_softmax[i][i]

```

so to do this we convert the lower triangular part equal infinity so that when we do softmax we get zero

In [30]:
print(x_batch[0])
print(y_batch[0])

tensor([26,  1, 30, 13, 36, 41, 34, 38])
tensor([ 1, 30, 13, 36, 41, 34, 38, 28])


In [31]:
torch.ones_like(attention_matrix[0])

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [32]:
torch.tril(torch.ones_like(attention_matrix[0]), diagonal=-1).bool() # should the diagonal also have no contribution?

tensor([[False, False, False, False, False, False, False, False],
        [ True, False, False, False, False, False, False, False],
        [ True,  True, False, False, False, False, False, False],
        [ True,  True,  True, False, False, False, False, False],
        [ True,  True,  True,  True, False, False, False, False],
        [ True,  True,  True,  True,  True, False, False, False],
        [ True,  True,  True,  True,  True,  True, False, False],
        [ True,  True,  True,  True,  True,  True,  True, False]])

In [33]:
attention_matrix[0].masked_fill(torch.tril(torch.ones_like(attention_matrix[0]), diagonal=-1).bool(), float('-inf'))

tensor([[-0.1206, -1.4007, -0.1955, -1.1356, -0.3318,  1.6046, -0.6190, -0.9474],
        [   -inf,  1.1576,  2.3776,  2.2315,  1.7283,  0.4289, -0.5666,  2.1490],
        [   -inf,    -inf, -1.6253,  0.5134,  0.9698,  0.4333, -0.0148,  0.3658],
        [   -inf,    -inf,    -inf,  1.8548,  3.0317,  2.0422, -0.4053,  1.8306],
        [   -inf,    -inf,    -inf,    -inf,  2.6862,  3.1695, -0.2330,  0.7469],
        [   -inf,    -inf,    -inf,    -inf,    -inf,  0.8786, -0.7581, -0.5808],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,  1.4286,  0.0312],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,  1.0234]],
       grad_fn=<MaskedFillBackward0>)

In [34]:
attention_matrix.masked_fill(torch.tril(torch.ones_like(attention_matrix), diagonal=-1).bool(), float('-inf'))[0]

tensor([[-0.1206, -1.4007, -0.1955, -1.1356, -0.3318,  1.6046, -0.6190, -0.9474],
        [   -inf,  1.1576,  2.3776,  2.2315,  1.7283,  0.4289, -0.5666,  2.1490],
        [   -inf,    -inf, -1.6253,  0.5134,  0.9698,  0.4333, -0.0148,  0.3658],
        [   -inf,    -inf,    -inf,  1.8548,  3.0317,  2.0422, -0.4053,  1.8306],
        [   -inf,    -inf,    -inf,    -inf,  2.6862,  3.1695, -0.2330,  0.7469],
        [   -inf,    -inf,    -inf,    -inf,    -inf,  0.8786, -0.7581, -0.5808],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,  1.4286,  0.0312],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,  1.0234]],
       grad_fn=<SelectBackward0>)

In [35]:
masked_attention_matrix = attention_matrix.masked_fill(torch.tril(torch.ones_like(attention_matrix), diagonal=-1).bool(), float('-inf'))

In [36]:
masked_attention_matrix[0]

tensor([[-0.1206, -1.4007, -0.1955, -1.1356, -0.3318,  1.6046, -0.6190, -0.9474],
        [   -inf,  1.1576,  2.3776,  2.2315,  1.7283,  0.4289, -0.5666,  2.1490],
        [   -inf,    -inf, -1.6253,  0.5134,  0.9698,  0.4333, -0.0148,  0.3658],
        [   -inf,    -inf,    -inf,  1.8548,  3.0317,  2.0422, -0.4053,  1.8306],
        [   -inf,    -inf,    -inf,    -inf,  2.6862,  3.1695, -0.2330,  0.7469],
        [   -inf,    -inf,    -inf,    -inf,    -inf,  0.8786, -0.7581, -0.5808],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,  1.4286,  0.0312],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,  1.0234]],
       grad_fn=<SelectBackward0>)

In [37]:
import torch.nn.functional as F


F.softmax(masked_attention_matrix[0], dim=-2)

tensor([[1.0000, 0.0719, 0.0697, 0.0182, 0.0162, 0.1186, 0.0657, 0.0168],
        [0.0000, 0.9281, 0.9136, 0.5263, 0.1268, 0.0366, 0.0693, 0.3708],
        [0.0000, 0.0000, 0.0167, 0.0944, 0.0594, 0.0368, 0.1203, 0.0623],
        [0.0000, 0.0000, 0.0000, 0.3611, 0.4670, 0.1837, 0.0814, 0.2697],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.3306, 0.5671, 0.0967, 0.0913],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0574, 0.0572, 0.0242],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5094, 0.0446],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1203]],
       grad_fn=<SoftmaxBackward0>)

### should the diagonal (i.e the current token) have any contribution towards update of input embedding vector?

if it does then ``V`[0] = V[0]`` and that would mean the updated embedding would become ``E_updated[0] = V[0] + E[0]`` need to think more about it

but according to my understanding `V[0]` is same as the absolute embedding of the token so `V[0] + E[0]` would be pracitically `2*absolute_mebdding[0]` since even `E[0]` was created any info about token after that so just itself, but since we don layer norm `norm(2*absolute_embedding[0]) = absolute_embedding[0]` so even if it scales a lot layer normalization brings down the scale of embedding vector so it fits with rest of embedding vector's scale

In [38]:
batch_attention_matrix = F.softmax(masked_attention_matrix, dim=-2)

In [39]:
batch_attention_matrix[0]

tensor([[1.0000, 0.0719, 0.0697, 0.0182, 0.0162, 0.1186, 0.0657, 0.0168],
        [0.0000, 0.9281, 0.9136, 0.5263, 0.1268, 0.0366, 0.0693, 0.3708],
        [0.0000, 0.0000, 0.0167, 0.0944, 0.0594, 0.0368, 0.1203, 0.0623],
        [0.0000, 0.0000, 0.0000, 0.3611, 0.4670, 0.1837, 0.0814, 0.2697],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.3306, 0.5671, 0.0967, 0.0913],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0574, 0.0572, 0.0242],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5094, 0.0446],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1203]],
       grad_fn=<SelectBackward0>)

In [40]:
V[0]

tensor([[-1.5340, -0.2835, -0.7389, -0.2533, -0.3472, -1.0250,  1.0312,  0.8763],
        [-0.0688, -1.0531, -0.5835,  0.4022,  0.2301, -0.4522, -0.6365, -0.0743],
        [ 0.1680,  0.6100, -1.8390,  0.6021, -1.5898, -0.7027, -0.4716,  0.1950],
        [-0.7219, -0.3745, -2.0341, -1.7686, -1.9071, -1.1831,  0.4164,  2.3593],
        [-0.7091, -0.4813, -0.6395, -0.0091, -0.0893, -0.9274, -1.0035,  1.2202],
        [ 0.2383, -0.4580,  0.3413,  1.2858, -0.2266, -0.0650, -1.4905, -0.8708],
        [ 0.1980,  0.3482,  0.6637,  0.5010,  0.8354, -0.0363,  0.0171,  0.0420],
        [-0.0649, -0.1889, -1.0767, -1.2216, -0.8833, -0.0987, -0.7100,  0.9288]],
       grad_fn=<SelectBackward0>)

In [41]:
batch_attention_matrix[0]@V[0]

tensor([[-1.5116e+00, -3.6581e-01, -8.9028e-01, -4.9794e-02, -4.6432e-01,
         -1.1547e+00,  7.5640e-01,  8.6221e-01],
        [-3.8183e-01, -7.4091e-01, -3.7141e+00, -3.7981e-01, -2.5319e+00,
         -1.8434e+00, -1.2464e+00,  1.8212e+00],
        [-7.8955e-02, -4.0500e-02, -2.3549e-01, -1.2613e-01, -1.7482e-01,
         -1.9144e-01, -1.2513e-01,  3.2948e-01],
        [-5.4944e-01, -4.6673e-01, -1.2069e+00, -6.9545e-01, -9.4226e-01,
         -9.0183e-01, -7.8207e-01,  1.5158e+00],
        [-8.6048e-02, -4.0240e-01, -5.1933e-02,  6.6308e-01, -1.5786e-01,
         -3.5593e-01, -1.2400e+00, -1.5936e-03],
        [ 2.3427e-02, -1.0930e-02,  3.1497e-02,  7.2871e-02,  1.3415e-02,
         -8.1924e-03, -1.0171e-01, -2.5086e-02],
        [ 9.7978e-02,  1.6896e-01,  2.9006e-01,  2.0073e-01,  3.8618e-01,
         -2.2897e-02, -2.2948e-02,  6.2845e-02],
        [-7.8084e-03, -2.2727e-02, -1.2955e-01, -1.4698e-01, -1.0628e-01,
         -1.1876e-02, -8.5425e-02,  1.1174e-01]], grad_fn=<MmBack

In [42]:
V_prime = batch_attention_matrix@V

In [43]:
embedding_unnormalized = V_prime + x_embeddings_total

# Layer normalization

we just normalize across the embedding dimension to make sure this addition does not scale the embedding vector to much across each layer so that it has the scale of the original embedding vector

In [44]:
mean_emebdding = embedding_unnormalized.mean(dim=-1)
mean_emebdding.shape

torch.Size([5, 8])

In [45]:
std_embedding = embedding_unnormalized.std(dim=-1)
std_embedding.shape

torch.Size([5, 8])

In [46]:
embedding_unnormalized.shape

torch.Size([5, 8, 8])

In [47]:
print(mean_emebdding[0][0], std_embedding[0][0])

tensor(-0.3098, grad_fn=<SelectBackward0>) tensor(1.7855, grad_fn=<SelectBackward0>)


In [51]:
(embedding_unnormalized[0][0] - mean_emebdding[0][0])/(std_embedding[0][0] + 0.0001)

tensor([-0.6958, -1.0390, -0.3248,  2.0772,  0.7325, -0.6856,  0.0256, -0.0901],
       grad_fn=<DivBackward0>)

In [49]:
embedding_normalized = (embedding_unnormalized - mean_emebdding.unsqueeze(-1))/(std_embedding.unsqueeze(-1) + 0.0001) # unsqueeze needed to broadcast
embedding_normalized[0][0]

tensor([-0.6958, -1.0390, -0.3248,  2.0772,  0.7325, -0.6856,  0.0256, -0.0901],
       grad_fn=<SelectBackward0>)

# Multi head self attention

Functionaly what multi headed attention does is, it employs multiple smaller self attention blocks, the input to each is the full embedding vectors, and the output are vectors with a smaller dimension

```

let's say the input is [6, 6, 6]  (here 6 represents the dimension of the embedding vector which goes in, and 3 is the context length)


[6, 6, 6] -> head 1 -> [2, 2, 2]'1 (here I am just using '1 as a label)

[6, 6, 6] -> head 2 -> [2, 2, 2]'2

[6, 6, 6] -> head 3 -> [2, 2, 2]'3


concat([2, 2, 2]'1, [2, 2, 2]'2, [2, 2, 2]'3) -> [6, 6, 6]'

[6, 6, 6]' -> linear layer -> [6, 6, 6]'' --> residual connection --> [6, 6, 6]'' + [6, 6, 6] --> layer norm --> [6, 6, 6]final

```



In [ ]:
# writing everything we have done in one block

# ----multi head repetition--------
d_k = d_q = 4
d_v = EMBEDDING_DIMENSION

W_q = nn.Linear(EMBEDDING_DIMENSION, d_q, bias = False)
W_k = nn.Linear(EMBEDDING_DIMENSION, d_k, bias = False)
W_v = nn.Linear(EMBEDDING_DIMENSION, d_v, bias = False) # for single head attention this needs to be same as embedding dimension cause the resulting vector of attention layer get added to input embedding vector so both need to have same dimension

Q, K, V = W_q(x_embeddings_total), W_k(x_embeddings_total), W_v(x_embeddings_total)

attention_matrix = Q@K.transpose(1, 2) # transpose the 1st and the 2nd dimension not the  this is equivalent to Q[i]@K[i].transpose where i is each element of the batch

masked_attention_matrix = attention_matrix.masked_fill(torch.tril(torch.ones_like(attention_matrix), diagonal=-1).bool(), float('-inf'))

batch_attention_matrix = F.softmax(masked_attention_matrix, dim=-2)

V_prime = batch_attention_matrix@V
# ----multi head repetition--------

embedding_unnormalized = V_prime + x_embeddings_total

mean_emebdding = embedding_unnormalized.mean(dim=-1)
std_embedding = embedding_unnormalized.std(dim=-1)

embedding_normalized = (embedding_unnormalized - mean_emebdding.unsqueeze(-1))/(std_embedding.unsqueeze(-1) + 0.0001) # unsqueeze needed to broadcast


In [ ]:
# let me create a class out of the repeated block

class SingleHead():

    def __init__(self, d_k, d_q, d_v) -> None:
        
        self.W_q = nn.Linear(EMBEDDING_DIMENSION, d_q, bias = False)
        self.W_k = nn.Linear(EMBEDDING_DIMENSION, d_k, bias = False)
        self.W_v = nn.Linear(EMBEDDING_DIMENSION, d_v, bias = False) # for single head attention this needs to be same as embedding dimension cause the resulting vector of attention layer get added to input embedding vector so both need to have same dimension
        pass